# Plot the SCvH EOS table

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
#from scipy import optimize
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

Convert units from cgs to SI:

In [ ]:
press_unit_si = 0.1        # erg/cm^3 in Pascal
spec_energy_unit_si = 1e4  # erg/g in J/kg

The SCvH equation of state (EOS) for hydrogen (H) and helium (He) is valid in the following region:
* Density: $-6.0 \leq \log(\rho) \leq 3.75$ (or $10^{-6} \leq \rho \leq 5.6234 \times 10^{3}$  g cm$^{-3}$), $\Delta \log(\rho) = 0.25$
* Tempeature: $2.10 \leq \log(T) \leq 7.06$ (or $125.89 \leq T \leq 1.1482 \times 10^7K$), $\Delta \log(T) = 0.08$

Check the original paper

The EOS tables were extended to lower temperatures (ref):
* Hydrogen: $10^{-6} \leq \rho \leq 10^{2}$ g cm$^{-3}$ and $10^2 \leq T \leq 10^6$ K



In [ ]:
#data = np.loadtxt("scvh_h_dt_cgs.csv", delimiter=",", skiprows=1)
data = np.loadtxt("scvh_hhe_y0.275_dt_cgs.csv", delimiter=",", skiprows=1) 

logT_table   = data[:, 0] 
logrho_table = data[:, 1]
logP_table   = data[:, 2]
logu_table   = data[:, 3]
logs_table   = data[:, 4]

#print(np.where(logrho_table == logrho_table[0]))

# All SCVH EOS tables obtained from Ravit have the same size
nRho = 201
nT   = 100

logrho_table = logrho_table[0:nRho]
logT_table = logT_table[0:np.size(logT_table):nRho]

logrho_min = np.min(logrho_table)
logrho_max = np.max(logrho_table)
logT_min   = np.min(logT_table)
logT_max   = np.max(logT_table)
    
dlogrho = logrho_table[1:]-logrho_table[:-1]
dlogT = logT_table[1:]-logT_table[:-1]

print("logrho_min = {:}".format(logrho_min))
print("logrho_max = {:}".format(logrho_max))
print("logT_min   = {:}".format(logT_min))
print("logT_max   = {:}".format(logT_max))
print()

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_table>=2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_table[index]))

# Split into arrays of constant T
logP_array = np.split(logP_table, nT)
logu_array = np.split(logu_table, nT)
logs_array = np.split(logs_table, nT)

# Generate 2d arrays
logP = np.vstack(logP_array)
logu = np.vstack(logu_array)
logs = np.vstack(logs_array)

In [ ]:
# Generate interpolation functions
logP_int = interp.interp2d(logrho_table, logT_table, logP, kind='linear')
logu_int = interp.interp2d(logrho_table, logT_table, logu, kind='linear')
logs_int = interp.interp2d(logrho_table, logT_table, logs, kind='linear')

In [ ]:
# Generate interpolation axis
nRho_int = 400
nT_int = 200

logrho_int = np.linspace(logrho_min, logrho_max, nRho_int)
logT_int = np.linspace(logT_min, logT_max, nT_int)

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 5
nSkipRho = 10

In [ ]:
# Plot log(P), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, nT, nSkipT):
    #ax[0][0].plot(logrho_table, logP_array[i], '-')
    ax[0][0].plot(logrho_table, logP[i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logP[:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logu[i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logu[:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logrho_table, logs[i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[2][1].plot(logT_table, logs[:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Plot log(P/rho), log(P/T), log(u/rho) and log(u/T)
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# P(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, logP[i,:]-logrho_table, '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P/rho) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logP[:,i]-logT_table, '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P/T) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logu[i,:]-logrho_table, '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u/rho) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logu[:,i]-logT_table, '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u/T) [erg g$^{-1}$]")


In [ ]:
def press_ideal_gas(rho, T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*rho*T

In [ ]:
def intenergy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/((gamma-1.0)*mu*m_H)*T

In [ ]:
def entropy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*(np.log(T/T0) - 1.0/(gamma-1.0)*np.log(rho/rho0))

In [ ]:
# Compare the pressure to an ideal gas
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# P(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, logP[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, np.log10(press_ideal_gas(10**logrho_table, 10**logT_table[i])), '--')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logP[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, np.log10(press_ideal_gas(10**logrho_table[i], 10**logT_table)), '--')
    
ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# P(rho, T=const) zoom
plt.gca().set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logP[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, np.log10(press_ideal_gas(10**logrho_table, 10**logT_table[i])), '--')
    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[1][0].set(title="SCvH EOS H (T=const.)")

ax[1][0].set_xlim(logrho_min, -3)
ax[1][0].set_ylim(0, 10)

# P(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logP[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, np.log10(press_ideal_gas(10**logrho_table[i], 10**logT_table)), '--')
    
ax[1][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[1][1].set(title="SCvH EOS H (rho=const.)")

ax[1][1].set_xlim(logT_min, 4)
ax[1][1].set_ylim(0, 12)



In [ ]:
# Compare the internal energy to an ideal gas
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# u(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT, 10):
    ax[0][0].plot(logrho_table, logu[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT, 10):
    ax[0][0].plot(logrho_table, np.ones(nRho)*np.log10(intenergy_ideal_gas(10**logT_table[i])), '--')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# u(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho, 10):
    ax[0][1].plot(logT_table, logu[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho, 10):
    ax[0][1].plot(logT_table, np.log10(intenergy_ideal_gas(10**logT_table)), '--')
    
ax[0][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const) zoom
plt.gca().set_prop_cycle(None)
for i in range(0, nT, 10):
    ax[1][0].plot(logrho_table, logu[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT, 10):
    ax[1][0].plot(logrho_table, np.ones(nRho)*np.log10(intenergy_ideal_gas(10**logT_table[i])), '--')
    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg cm$^{-3}$]")
ax[1][0].set(title="SCvH EOS H (T=const.)")

ax[1][0].set_xlim(logrho_min, -4.0)
ax[1][0].set_ylim(8.6144, 12.3542)

# u(rho=const, T) zoom
plt.gca().set_prop_cycle(None)
for i in range(0, nRho, 10):
    ax[1][1].plot(logT_table, logu[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho, 10):
    ax[1][1].plot(logT_table, np.log10(intenergy_ideal_gas(10**logT_table)), '--')
    
ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg cm$^{-3}$]")
ax[1][1].set(title="SCvH EOS H (rho=const.)")

ax[1][1].set_xlim(logT_min, 3.5)
ax[1][1].set_ylim(8.6192, 12.3540)

Now we limit the EOS to the region of interest.

In [ ]:
# Limit the data to the region in which we are interested
logrho_grid_min = logrho_min
logrho_grid_max = -4.0

logT_grid_min = logT_min
logT_grid_max = 3.5

print("logrho_grid_min= {:} rho_grid_min= {:}".format(logrho_grid_min, 10**logrho_grid_min))
print("logrho_grid_max= {:} rho_grid_max= {:}".format(logrho_grid_max, 10**logrho_grid_max))
print("logT_grid_min= {:} T_grid_min= {:}".format(logT_grid_min, 10**logT_grid_min))
print("logT_grid_max= {:} T_grid_max= {:}".format(logT_grid_max, 10**logT_grid_max))
print()

index_logrho_grid = np.where(logrho_table < logrho_grid_max)[0]
index_logT_grid = np.where(logT_table < logT_grid_max)[0]

nRho_grid = np.size(index_logrho_grid)
nT_grid = np.size(index_logT_grid)
print("nRho= {:} nT= {:}".format(nRho_grid, nT_grid))

logrho_grid = logrho_table[index_logrho_grid]
logT_grid = logT_table[index_logT_grid]

logP_grid = logP[:nT_grid,:nRho_grid]
logu_grid = logu[:nT_grid,:nRho_grid]
logs_grid = logs[:nT_grid,:nRho_grid]

print(logP.shape)
print(logP_grid.shape)
#logP_grid = logP[index_logT_grid, index_logrho_grid]
#logu_grid = logu[index_logT_grid, index_logrho_grid]


In [ ]:
# Generate interpolation functions
logP_grid_int = interp.interp2d(logrho_grid, logT_grid, logP_grid, kind='linear')
logu_grid_int = interp.interp2d(logrho_grid, logT_grid, logu_grid, kind='linear')
logs_grid_int = interp.interp2d(logrho_grid, logT_grid, logs_grid, kind='linear')

In [ ]:
# Generate interpolation axis
nRho_grid_int = 400
nT_grid_int = 400

logrho_grid_int = np.linspace(logrho_grid_min, logrho_grid_max, nRho_grid_int)
logT_grid_int = np.linspace(logT_grid_min, logT_grid_max, nT_grid_int)

In [ ]:
# Plot the reduced grid
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# P(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[0][0].plot(logrho_grid, logP_grid[i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 8):
    ax[0][1].plot(logT_grid, logP_grid[:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[1][0].plot(logrho_grid, logu_grid[i,:], '-')
    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")
ax[1][0].set(title="SCvH EOS H (T=const.)")

# u(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 8):
    ax[1][1].plot(logT_grid, logu_grid[:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")
ax[1][1].set(title="SCvH EOS H (rho=const.)")


In [ ]:
# Compare the reduced grid to an ideal gas
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# P(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[0][0].plot(logrho_grid, logP_grid[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[0][0].plot(logrho_grid, np.log10(press_ideal_gas(10**logrho_grid, 10**logT_grid[i])), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 8):
    ax[0][1].plot(logT_grid, logP_grid[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 8):
    ax[0][1].plot(logT_grid, np.log10(press_ideal_gas(10**logrho_grid[i], 10**logT_grid)), '--')

    
ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[1][0].plot(logrho_grid, logu_grid[i,:], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nT_grid, 4):
    ax[1][0].plot(logrho_grid, np.ones(nRho_grid)*np.log10(intenergy_ideal_gas(10**logT_grid[i])), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")
ax[1][0].set(title="SCvH EOS H (T=const.)")


# u(rho=const, T)
plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 4):
    ax[1][1].plot(logT_grid, logu_grid[:,i], '-')

plt.gca().set_prop_cycle(None)
for i in range(0, nRho_grid, 4):
    ax[1][1].plot(logT_grid, np.log10(intenergy_ideal_gas(10**logT_grid)), '--')

ax[1][1].get_xlim()
ax[1][1].get_ylim()

#ax[1][0].get_ylim()


In [ ]:
# Plot the difference in pressure on a rho x T grid
P_scvh = 10**logP_grid
P_ig = list()

for i in range(0, nT_grid):
    P_ig.append(press_ideal_gas(10**logrho_grid, 10**logT_grid[i]))

P_ig_array = np.vstack(P_ig)
#P_ig = np.vstack(for i in range(0, nT_grid): press_ideal_gas(10**logrho_grid, 10**logT_grid[i]))

print(P_scvh.shape)
print(P_ig_array.shape)

P_err = np.abs(P_scvh-P_ig_array)/P_scvh

diff = np.zeros(P_err.shape)

diff[np.where(P_err >= 0)] = 4
diff[np.where(P_err <= 1)] = 3
diff[np.where(P_err < 1e-1)] = 2
diff[np.where(P_err < 1e-2)] = 1
diff[np.where(P_err < 1e-3)] = 0

fig, ax = plt.subplots()

plt.imshow(diff, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-3}$", "$<10^{-2}$", "$<10^{-1}$", "$<10^{0}$", "$\geq 10^{0}$"])


plt.xlabel("Density [g cm$^3$]")
plt.ylabel("Temperature [K]")


In [ ]:
# Plot the difference in internal energy on a rho x T grid
u_scvh = 10**logu_grid
u_ig = list()

for i in range(0, nT_grid):
    u_ig.append(intenergy_ideal_gas(10**logrho_grid, 10**logT_grid[i]))

u_ig_array = np.vstack(u_ig)

print(u_scvh.shape)
print(u_ig_array.shape)

u_err = np.abs(u_scvh-u_ig_array)/u_scvh

diff = np.zeros(u_err.shape)

diff[np.where(u_err >= 0)] = 4
diff[np.where(u_err < 1)] = 3
diff[np.where(u_err < 1e-1)] = 2
diff[np.where(u_err < 1e-2)] = 1
diff[np.where(u_err < 1e-3)] = 0

fig, ax = plt.subplots()

plt.imshow(diff, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-3}$", "$<10^{-2}$", "$<10^{-1}$", "$<10^{0}$", "$\geq 10^{0}$"])


plt.xlabel("Density [g cm$^3$]")
plt.ylabel("Temperature [K]")


In [ ]:
# Plot the difference in pressure on a rho x T grid
P_scvh = 10**logP_grid_int(logrho_grid_int, logT_grid_int)
P_ig = list()

for i in range(0, nT_grid_int):
    P_ig.append(press_ideal_gas(10**logrho_grid_int, 10**logT_grid_int[i]))

P_ig_array = np.vstack(P_ig)
#P_ig = np.vstack(for i in range(0, nT_grid): press_ideal_gas(10**logrho_grid, 10**logT_grid[i]))

print(P_scvh.shape)
print(P_ig_array.shape)

P_err = np.abs(P_scvh-P_ig_array)/P_scvh

diff = np.zeros(P_err.shape)

diff[np.where(P_err >= 0)] = 4
diff[np.where(P_err <= 1)] = 3
diff[np.where(P_err < 1e-1)] = 2
diff[np.where(P_err < 1e-2)] = 1
diff[np.where(P_err < 1e-3)] = 0

fig, ax = plt.subplots()

plt.imshow(diff, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-3}$", "$<10^{-2}$", "$<10^{-1}$", "$<10^{0}$", "$\geq 10^{0}$"])


plt.xlabel("Density [g cm$^3$]")
plt.ylabel("Temperature [K]")


In [ ]:
# Plot the difference in internal energy on a rho x T grid
u_scvh = 10**logu_grid_int(logrho_grid_int, logT_grid_int)
u_ig = list()

for i in range(0, nT_grid_int):
    u_ig.append(intenergy_ideal_gas(10**logrho_grid_int, 10**logT_grid_int[i]))

u_ig_array = np.vstack(u_ig)

print(u_scvh.shape)
print(u_ig_array.shape)

u_err = np.abs(u_scvh-u_ig_array)/u_scvh

diff = np.zeros(u_err.shape)

diff[np.where(u_err >= 0)] = 4
diff[np.where(u_err < 1)] = 3
diff[np.where(u_err < 1e-1)] = 2
diff[np.where(u_err < 1e-2)] = 1
diff[np.where(u_err < 1e-3)] = 0

fig, ax = plt.subplots()

plt.imshow(diff, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-3}$", "$<10^{-2}$", "$<10^{-1}$", "$<10^{0}$", "$\geq 10^{0}$"])


plt.xlabel("Density [g cm$^3$]")
plt.ylabel("Temperature [K]")


In [ ]:
# Plot the difference in pressure and internal energy on a rho x T grid
#delta_P = (10**logP_grid_int(logrho_grid_int, logT_grid_int)-10**press_ideal_gas(logrho_grid_int, logT_grid_int))/10**logP_grid_int(logrho_grid_int, logT_grid_int)

#delta_P = 10**logP_grid_int(logrho_grid_int, logT_grid_int)
#delta_P = 10**press_ideal_gas(logrho_grid_int, logT_grid_int)

P_scvh = 10**logP_grid
print(P_scvh.shape)

P_ig = list()

for i in range(0, nT_grid):
    P_ig.append(press_ideal_gas(10**logrho_grid, 10**logT_grid[i]))
    
P_ig_array = np.vstack(P_ig)
#P_ig = np.vstack(for i in range(0, nT_grid): press_ideal_gas(10**logrho_grid, 10**logT_grid[i]))

print(P_ig_array.shape)

P_err = np.abs(P_scvh-P_ig_array)/P_scvh

diff = np.zeros(P_err.shape)

diff[np.where(P_err >= 0)] = 4
diff[np.where(P_err >= 1)] = 3
diff[np.where(P_err < 1)] = 2
diff[np.where(P_err < 1e-1)] = 1
diff[np.where(P_err < 1e-3)] = 0

fig, ax = plt.subplots()

plt.imshow(diff, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-10}$", "$<10^{-6}$", "$<10^{-3}$", "$<10^{-1}$", "$\geq 10^{-1}$"])


plt.xlabel("Density [g cm$^3$]")
plt.ylabel("Temperature [K]")
